In [8]:
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install tiktoken

In [9]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import sklearn as sk
import sklearn.model_selection

import tiktoken

nltk.download('punkt')
nltk.download('stopwords')

def tokenizer(X):
    stop_words = set(stopwords.words('english'))
    num_tokens = 0
    num_sentences = 0
    tokenized_documents = []
    for text in X:
        sentences = sent_tokenize(text)
        num_sentences += len(sentences)
        tokenized_sentences = [nltk.RegexpTokenizer(r"\w+").tokenize(s) for s in sentences]
        tokenized_document = [word.lower() for sentence in tokenized_sentences for word in sentence if word.lower() not in stop_words]
        num_tokens += len(tokenized_document)
        tokenized_documents.append(tokenized_document)
    print("Number of tokens: ", num_tokens)
    print("Number of sentences: ", num_sentences)
    return tokenized_documents

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import sklearn as sk

# tf-idf vectorizer
def vectorizer(tokenized_documents):
    ## tokenized_documents is a list of lists, where each inner list contains tokens
    vectorizer = sk.feature_extraction.text.TfidfVectorizer(lowercase=False, preprocessor=None, tokenizer=lambda x: x)
    X = vectorizer.fit_transform(tokenized_documents)
    return X, vectorizer

# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [11]:
import sys
# sys.path.append('../')
import numpy as np
import pandas as pd
import sklearn as sk

In [12]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [13]:
# FOR TESTING : only select first 20000 samples
# X, y = X[:20000], y[:20000]

In [14]:
tokenized_documents = tokenizer(X)

Number of tokens:  23767229
Number of sentences:  2832806


In [15]:
X, vect = vectorizer(tokenized_documents)

/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [16]:
# most frequent words
print("Top 10 most frequent words in the dataset")
print(vect.get_feature_names_out()[:10])

# least frequent words
print("Top 10 least frequent words in the dataset")
print(vect.get_feature_names_out()[-10:])

Top 10 most frequent words in the dataset
['0' '00' '000' '0000' '000001' '00001' '000013' '0000soo' '0001'
 '000111052']
Top 10 least frequent words in the dataset
['¾' 'â' 'çay' 'çaykur' 'çelem' 'être' 'île' 'ît' 'ø' 'þ']


In [17]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(454763, 120144) (113691, 120144) (454763,) (113691,)
  (0, 94340)	0.2698393204452672
  (0, 107461)	0.356573063664095
  (0, 60888)	0.6837599652000029
  (0, 31216)	0.37896669799521565
  (0, 25542)	0.19195720719400375
  (0, 79539)	0.28524356089653274
  (0, 69704)	0.22960104461763894
  (0, 66863)	0.13405538070127823


# Recurrent Neural Network
### Model starts here

In [19]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

def batch_generator(X, y, batch_size=32, num_classes=None, timesteps=1):
    num_samples = X.shape[0]
    features_per_timestep = X.shape[1] // timesteps
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X[start:end].toarray()  # Convert only this slice to dense
            X_batch = X_batch.reshape(-1, timesteps, features_per_timestep)  # Reshape for LSTM
            y_batch = to_categorical(y[start:end], num_classes=num_classes)
            yield (X_batch, y_batch)


# Setup the model
model = Sequential([
    LSTM(512, input_shape=(None, X_train.shape[1]), activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(256, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(128, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(np.max(y_train) + 1, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model using a generator
# Assuming `timesteps` is set to 1, you might want to adjust this based on how you wish to frame your sequences
train_generator = batch_generator(X_train, y_train, batch_size=32, num_classes=np.max(y_train) + 1, timesteps=1)
model.fit(train_generator, epochs=20, steps_per_epoch=int(np.ceil(X_train.shape[0] / 32)))

2024-05-02 12:36:06.373627: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78746 MB memory:  -> device: 0, name: NVIDIA H100 PCIe, pci bus id: 0000:b5:00.0, compute capability: 9.0
/opt/mamba/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


I0000 00:00:1714653371.021265    6597 service.cc:145] XLA service 0x5598093c4220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714653371.021334    6597 service.cc:153]   StreamExecutor device (0): NVIDIA H100 PCIe, Compute Capability 9.0
2024-05-02 12:36:11.095366: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-02 12:36:11.446551: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
I0000 00:00:1714653375.836527    6922 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_88', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1714653375.935112    6926 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_89', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1714653375.

   12/14212 ━━━━━━━━━━━━━━━━━━━━ 3:50 16ms/step - accuracy: 0.5049 - loss: 1.7675 

I0000 00:00:1714653380.624627    6597 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14208/14212 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7077 - loss: 0.8059

I0000 00:00:1714653603.992668    7622 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_74', 24 bytes spill stores, 24 bytes spill loads



14212/14212 ━━━━━━━━━━━━━━━━━━━━ 245s 16ms/step - accuracy: 0.7077 - loss: 0.8058
Epoch 2/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 231s 16ms/step - accuracy: 0.8068 - loss: 0.5375
Epoch 3/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 216s 15ms/step - accuracy: 0.8736 - loss: 0.3593
Epoch 4/20
 2338/14212 ━━━━━━━━━━━━━━━━━━━━ 2:57 15ms/step - accuracy: 0.9050 - loss: 0.2710

In [ ]:
test_generator = batch_generator(X_test, y_test, batch_size=32, num_classes=np.max(y_train) + 1, timesteps=1)

In [ ]:
# Evaluate the model
predictions_prob = model.predict(test_generator, steps=int(np.ceil(X_test.shape[0] / 32)))
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
from sklearn import metrics
print(metrics.confusion_matrix(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

   8/3553 ━━━━━━━━━━━━━━━━━━━━ 1:05 19ms/step

3553/3553 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step
[[ 7908   770   509   244   895]
 [ 1157  2953   683   277   785]
 [  836   612  4536   899  1602]
 [  568   309  1023  8507  5716]
 [ 1310   537  1249  3007 66799]]
              precision    recall  f1-score   support

           1       0.67      0.77      0.72     10326
           2       0.57      0.50      0.54      5855
           3       0.57      0.53      0.55      8485
           4       0.66      0.53      0.59     16123
           5       0.88      0.92      0.90     72902

    accuracy                           0.80    113691
   macro avg       0.67      0.65      0.66    113691
weighted avg       0.79      0.80      0.79    113691



In [ ]:
# Save the model with current date and time in model folder
import datetime

# Create a folder named _models in the current directory
import os
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/config1_rnn_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

In [ ]:
# test the model
test_sentences = [
    'This is a good product',
    'This is a bad product',
    'This is a product',
    'This is a very good product',
    'This is a very bad product',
    'That was bad'
]

for sentence in test_sentences:
    test_tokenized = tokenizer([sentence])
    test_vec = vect.transform(test_tokenized)
    test_vec_dense = test_vec.toarray()  # Convert sparse tensor to dense tensor
    test_vec_lstm = test_vec_dense.reshape(-1, timesteps, features_per_timestep)
    result = model.predict(test_vec_lstm)
    predicted_score = result.argmax() + 1
    print(f"Test sentence: {sentence}")
    print(f"Predicted score: {predicted_score}")
    print()

Number of tokens:  2
Number of sentences:  1


NameError: name 'timesteps' is not defined